In [1]:
import os
from ollama import Client, ResponseError
import pandas as pd
import numpy as np
import time
import struct

In [2]:
HUMAN_TEXT_DS = 'human-written-text-dataset.csv'
AI_FOLDER = 'ai'
GPT = 'gpt-oss:120b'
GEMINI = 'gemini-3-flash-preview:cloud'
COGITO = 'cogito-2.1:671b-cloud'
NUM_MODELS = 2

In [3]:
df = pd.read_csv(HUMAN_TEXT_DS)
df

,index,url,text,origin
0,0,UNK,العالم يحتفي بالفنون المغربية في دولة الإمارات...,ANAD
1,1,UNK,ميلانو.. أوبرا تسلط الضوء على قضية التغيير الم...,ANAD
2,2,UNK,"الليلة.. تعود مؤامرات ""الحرملك"" في #سرايا عابد...",ANAD
3,3,UNK,منى واصف: أفضل الأدوار التي أصفع فيها زملائي \...,ANAD
4,4,UNK,هكذا ظهر عمرو دياب بحفل اليونان.. ودينا الشربي...,ANAD
...,...,...,...,...
6995,6995,UNK,"البراكين تخيف السياح من السفر لـ""بالي"" \n\nمس...",ANAD
6996,6996,UNK,حجز 90% من رحلات روسيا الجوية للقاهرة \n\nعضو ...,ANAD
6997,6997,UNK,الخطوط السعودية تبيع تذاكر إلكترونية بـ2.7 ملي...,ANAD
6998,6998,UNK,إياتا: الطلب العالمي على النقل الجوي يرتفع 3.4...,ANAD


In [4]:
human_text = df['text']
human_text

0       العالم يحتفي بالفنون المغربية في دولة الإمارات...
1       ميلانو.. أوبرا تسلط الضوء على قضية التغيير الم...
2       الليلة.. تعود مؤامرات "الحرملك" في #سرايا عابد...
3       منى واصف: أفضل الأدوار التي أصفع فيها زملائي \...
4       هكذا ظهر عمرو دياب بحفل اليونان.. ودينا الشربي...
                              ...                        
6995    البراكين تخيف السياح من السفر لـ"بالي"  \n\nمس...
6996    حجز 90% من رحلات روسيا الجوية للقاهرة \n\nعضو ...
6997    الخطوط السعودية تبيع تذاكر إلكترونية بـ2.7 ملي...
6998    إياتا: الطلب العالمي على النقل الجوي يرتفع 3.4...
6999    تعيين رئيس تنفيذي جديد للخطوط التركية \n\nالخط...
Name: text, Length: 7000, dtype: object

In [5]:
parts = np.array_split(human_text, NUM_MODELS)
print(f'Number of parts: {len(parts)}')
for i, part in enumerate(parts):
    part.index = pd.RangeIndex(start=0, stop=len(part), step=1)
    print(f'Size of part {i}: {len(part)}', '\n')
    print(part[:3], '\n')

Number of parts: 2
Size of part 0: 3500 

0    العالم يحتفي بالفنون المغربية في دولة الإمارات...
1    ميلانو.. أوبرا تسلط الضوء على قضية التغيير الم...
2    الليلة.. تعود مؤامرات "الحرملك" في #سرايا عابد...
Name: text, dtype: object 

Size of part 1: 3500 

0    مصر.. مستشار وزير تقاضى رشوة بمليون جنيه \n\nه...
1    طالبان تعلن السيطرة على بنجشير.. وتحتفل في كاب...
2    أميركا ترفض تعليقات وفد النظام على انسحاب معار...
Name: text, dtype: object 



d:\Artificial Intelligence Master\Master 2\Final Project\StartUp\Dataset Building\.env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [6]:
client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

"""
prompt = lambda input: f'''
تعريف النسق المربّع:

هو أسلوب يُستخدم لإضافة نصوص جديدة داخل المقالات أو النصوص الأصلية دون حذف أو تعديل أي كلمة منها،

وتكون الإضافات على شكل فقرات عددها من 1 إلى 5 توضع بين [أقواس مربعة]، موزّعة في بدايات ووسط ونهايات الفقرات،

بنسبة لا تتجاوز 25٪ من حجم النص الكلي،

وبنفس لغة النص الأصلي (عربية أو إنجليزية).

طبق النسق المربع على النص التالي و لتكن إجابتك هي النص الجديد فقط دون كلمات إضافية كي أنسخه مباشرة:
{input}
'''
"""

prompt = lambda input: f'''
المطلوب منك هو إدخال نصوص جديدة داخل المقال المرفق دون تعديل أو حذف أي كلمة منه. يجب أن تكون الإضافات على شكل فقرات عددها من 3 إلى 5، موزّعة في بدايات ووسط ونهايات الفقرات بنسبة لا تتجاوز 25٪ من حجم المقال المرفق. ضع هذه الإضافات بين [أقواس مربعة]، واكتبها بنفس لغة النص الأصلي (العربية الفصحى). يجب أن تكون الإجابة هي المقال كاملاً بعد إدخال النصوص الجديدة، بحيث يمكنني نسخه مباشرة. اجعل الإضافات منسجمة مع الأسلوب الأصلي بحيث يصعب التفريق بينها وبين النص الأصلي.
و ها هو المقال:
{input}
'''



In [7]:
def fetch_ai_edited_text(model, part, model_name:str, sleep_duration=1):
    # ensure all the necessary folders do exist
    if not os.path.isdir(AI_FOLDER):
       os.makedirs(AI_FOLDER, exist_ok=True)
    if not os.path.isdir(model_name):
       os.makedirs(AI_FOLDER + '/' + model_name, exist_ok=True)
    
    # the path to store ai edited texts
    PATH = AI_FOLDER + '/' + model_name + '/'

    # the path to the file that contains the index at which fetching is stopped last time
    index_path = PATH + 'stopped_at.bin'

    # if index file does not exist, create it and initialize the index to 0. else, retrieve the index
    index = 0
    if not os.path.exists(index_path):
       with open(index_path, 'wb') as file:
          file.write(struct.pack('i', 0))
    else:
       with open(index_path, 'rb') as file:
          bin_data = file.read(4)
          index = struct.unpack('i', bin_data)[0]
    
    # specify where to start
    start = 0 if index == 0 else index + 1

    for i in range(start, len(part)):
        messages = [
            {
              'role': 'user',
              'content': prompt(part[i]),
            },
        ]

        with open(f'{PATH}{model_name.lower()}_{i}.txt', 'w', encoding='utf-8') as file:
          try:
            for p in client.chat(model, messages=messages, stream=True):
              file.write(p['message']['content'])
            print(f'{PATH}{model_name.lower()}_{i}.txt has been created.')
          except ResponseError as e:
              print(e)
              break
          
        with open(index_path, 'wb') as file:
           file.write(struct.pack('i', i))
        
        time.sleep(sleep_duration)

In [8]:
fetch_ai_edited_text(GPT, parts[0], 'GPT', sleep_duration=0.1)

In [9]:
fetch_ai_edited_text(COGITO, parts[1], 'COGITO', sleep_duration=0.1)